# Thermography

## Libraries

In [271]:
import os
import pandas as pd

from sklearn.model_selection import train_test_split

## Data selection

In [272]:
class Data:
	def __init__(self, root):
		self.table = self.__generate_table(root)
		self.training, self.validation, self.test = None, None, None

	def __generate_table(self, root):
		files = []

		for category in os.listdir(root):
			for filename in os.listdir(root + category):
				files.append([root + filename, category])

		return pd.DataFrame(files, columns=["filename", "category"])

In [273]:
data = Data("./data/thermography/")

## Preprocessing and transformation

In [274]:
data.table.category = data.table.category.factorize()[0]

In [275]:
data.training, data.test = train_test_split(data.table, test_size=0.15, shuffle=True)

In [276]:
print(f"Raw: {data.table.category.value_counts().values}")

print(f"Training: {data.training.category.value_counts().values}")
print(f"Test: {data.test.category.value_counts().values}")

Raw: [99 96]
Training: [84 81]
Test: [15 15]
